<a href="https://colab.research.google.com/github/whoami-Lory271/DL-project/blob/main/LSTM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install pytorch-lightning --quiet
!pip install torchmetrics --quiet

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 825.8/825.8 KB 25.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 517.2/517.2 KB 42.3 MB/s eta 0:00:00


In [ ]:
from numpy import array
from sklearn.preprocessing import OneHotEncoder
import math
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.nn.modules.normalization import LayerNorm
import pickle
import os
from torch.utils.data import Dataset, DataLoader
from torchvision.transforms import ToTensor
from pathlib import Path
import pandas as pd
import numpy as np
import csv
import torchmetrics
import pytorch_lightning as pl
from pytorch_lightning.callbacks.progress import TQDMProgressBar

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


# Helper Function

In [ ]:
def create_padding(encode, vocab, max_len):
    enc = np.array(encode)
    encoded = np.zeros(max_len + 2)
    for k in range(len(enc)):
      encoded[k] = enc[k]
    return encoded

In [ ]:
def get_padding_mask(X,inverse=False):
  pad = None
  if inverse == False:
    pad = X == 0
  else:
    pad = X != 0
  padding_mask = pad.repeat(1,1,X.shape[1]).reshape((X.shape[0],X.shape[1],X.shape[1]))
  if inverse == False:
    padding_mask[pad] = True
  return padding_mask

#Preprocessing

In [ ]:
save_path = "/content/drive/MyDrive/Deep learning" #change if needed

Path for easy train data:

In [ ]:
#take all the txt files
easy_path = os.path.join(save_path,"easy_pickle")
easy_texts_path=[str(path) for path in Path(easy_path).glob("*.txt")]
print(easy_texts_path)

['/content/drive/MyDrive/Deep learning/easy_pickle/algebra__linear_1d_easy.txt', '/content/drive/MyDrive/Deep learning/easy_pickle/calculus__differentiate_easy.txt', '/content/drive/MyDrive/Deep learning/easy_pickle/probability__swr_p_level_set_easy.txt']


Path for medium train data:

In [ ]:
medium_path= os.path.join(save_path,"medium_pickle")
medium_texts_path=[str(path) for path in Path(medium_path).glob("*.txt")]
print(medium_texts_path)

['/content/drive/MyDrive/Deep learning/medium_pickle/algebra__linear_1d_medium.txt', '/content/drive/MyDrive/Deep learning/medium_pickle/calculus__differentiate_medium.txt', '/content/drive/MyDrive/Deep learning/medium_pickle/probability__swr_p_level_set_medium.txt']


Path for hard train data:

In [ ]:
hard_path= os.path.join(save_path,"hard_pickle")
hard_texts_path=[str(path) for path in Path(hard_path).glob("*.txt")]
print(hard_texts_path)

['/content/drive/MyDrive/Deep learning/hard_pickle/algebra__linear_1d_hard.txt', '/content/drive/MyDrive/Deep learning/hard_pickle/calculus__differentiate_hard.txt', '/content/drive/MyDrive/Deep learning/hard_pickle/probability__swr_p_level_set_hard.txt']


Path for interpolate data:

In [ ]:
interpolate_path= os.path.join(save_path,"interpolate_pickle")
interpolate_texts_path=[str(path) for path in Path(interpolate_path).glob("*.txt")]
print(interpolate_texts_path)

['/content/drive/MyDrive/Deep learning/interpolate_pickle/algebra__linear_1d.txt', '/content/drive/MyDrive/Deep learning/interpolate_pickle/calculus__differentiate.txt', '/content/drive/MyDrive/Deep learning/interpolate_pickle/probability__swr_p_level_set.txt']


In [ ]:
def create_vocabulary_from_directory(sentences_path):
  vocabulary = {}
  vocabulary['<sos>'] = 1
  vocabulary['<eos>'] = 2
  vocabulary['<pad>'] = 0
  index = 3
  conta = 0
  for file in sentences_path:
    with open(file, 'r') as file_in:
      text = file_in.read()
      sentences = text.split('\n')
      sentences_new = sentences[:-1] #remove last element because it's empty
      print("num sentences", len(sentences_new))
      for s in sentences_new:
        conta +=1
        for t in s:
         if t not in vocabulary:
            vocabulary[t] = index
            index += 1
      file_in.close()
  return vocabulary, conta

In [ ]:
# vocabulary, conta = create_vocabulary_from_directory(ft_texts_path)
# print(conta)

In [ ]:
# #create the pickle
# outfile = open(os.path.join(save_path,"vocabs"), 'wb')
# pickle.dump(vocabulary, outfile)
# outfile.close()

In [ ]:
def encoding_from_sentence(vocab, sentence):
  enc= [1]
  for token in sentence:
    enc.append(vocab[token])
  enc.append(2)
  return enc

In [ ]:
def create_pickles(vocabulary, sentences_paths, column_names=["Encoding"], path_quest="/content/drive/MyDrive/Deep learning/questions", path_answ="/content/drive/MyDrive/Deep learning/answers"):
  quest = []
  answ = []
  out_file_quest = open(path_quest, 'wb')
  out_file_answ = open(path_answ, 'wb')
  for file in sentences_paths:
    with open(file, 'r') as file_in:
      text = file_in.read()
      sentences = text.split('\n')
      sentences = sentences[:-1]
      print("num sentences", len(sentences))
      for i, sentence in enumerate(sentences):
          enc = encoding_from_sentence(vocabulary, sentence)
          if i % 2 == 0: #question
            #quest.append({"Sentence": sentence, "Encoding": enc})
            quest.append({"Encoding": enc})
          elif i % 2 == 1: #answer
            #answ.append({"Sentence": sentence, "Encoding": enc})
            answ.append({"Encoding": enc})
  pickle.dump(quest, out_file_quest)
  pickle.dump(answ, out_file_answ)
  out_file_quest.close()
  out_file_answ.close()
  return path_quest, path_answ

##Dataset

In [ ]:
#unpickle the vocabulary
infile = open(os.path.join(save_path,'vocabs'),'rb')
vocab = pickle.load(infile)
infile.close()

Execute if you want to train easy data:

In [ ]:
#unpickle the qestions
with open(os.path.join(easy_path,"questions"),'rb') as infile:
  quest = pickle.load(infile)
print(len(quest))

#unpickle the answers
with open(os.path.join(easy_path,"answers"),'rb') as infile:
  answ = pickle.load(infile)
print(len(answ))

1999998
1999998


Execute if you want to train medium data:

In [ ]:
#unpickle the qestions
with open(os.path.join(medium_path,"questions"),'rb') as infile:
  quest = pickle.load(infile)
print(len(quest))

#unpickle the answers
with open(os.path.join(medium_path,"answers"),'rb') as infile:
  answ = pickle.load(infile)
print(len(answ))

Execute if you want to train hard data:

In [ ]:
#unpickle the qestions
with open(os.path.join(hard_path,"questions"),'rb') as infile:
  quest = pickle.load(infile)
print(len(quest))

#unpickle the answers
with open(os.path.join(hard_path,"answers"),'rb') as infile:
  answ = pickle.load(infile)
print(len(answ))

Execute to load test:

In [ ]:
#unpickle the qestions
with open(os.path.join(interpolate_path,"questions"),'rb') as infile:
  test_quest = pickle.load(infile)
print(len(test_quest))

#unpickle the answers
with open(os.path.join(interpolate_path,"answers"),'rb') as infile:
  test_answ = pickle.load(infile)
print(len(test_answ))

30000
30000


In [ ]:
class MathematicsDataset(Dataset):
  def __init__(self, quest, answ, vocab , max_len_quest=160, max_len_answ=30):
    self.quest = quest
    self.answ = answ
    self.vocab = vocab
    self.max_len_quest = max_len_quest
    self.max_len_answ = max_len_answ

  def __len__(self):
    return len(self.quest)

  def __getitem__(self, idx):
    assert(idx  < len(self.quest)) #indices should start from 0 to len - 1 (there are 5999994 elements)
    question = self.quest[idx]["Encoding"]
    encoding1 = torch.from_numpy(create_padding(question, self.vocab, self.max_len_quest))
    encoding1 = encoding1.type(torch.int64)
    #self.sample["Question"] = encoding
      
    answer = self.answ[idx]["Encoding"]
    encoding2 = torch.from_numpy(create_padding(answer, self.vocab, self.max_len_answ))
    encoding2 = encoding2.type(torch.int64)
    #self.sample["Answer"] = encoding
      
    return encoding1, encoding2 

In [ ]:
class PlMathematicsDataset(pl.LightningDataModule):
    def __init__(self, train_quest, train_answ, vocab , test_quest, test_answ, max_len_quest=160, max_len_answ=30, batch_size=128):
        super().__init__()
        self.train_quest = train_quest
        self.train_answ = train_answ
        self.test_quest = test_quest
        self.test_answ = test_answ
        self.vocab = vocab
        self.max_len_quest = max_len_quest
        self.max_len_answ = max_len_answ
        self.batch_size = batch_size
        self.dataset = MathematicsDataset(self.train_quest, self.train_answ, self.vocab)

    # def prepare_data(self):
        # tok, load, ecc...
        # MNIST(self.data_dir, train=True, download=True)
        # MNIST(self.data_dir, train=False, download=True)

    def setup(self, stage=None):

        # Assign train/val datasets for use in dataloaders
        if stage == "fit" or stage is None:
            self.train_ds = MathematicsDataset(self.train_quest, self.train_answ, self.vocab)

        # Assign test dataset for use in dataloader(s)
        if stage == "test" or stage is None:
            self.test_ds = MathematicsDataset(self.test_quest, self.test_answ, self.vocab)

        if stage == "predict" or stage is None:
            self.predict_ds = MathematicsDataset(self.test_quest, self.test_answ, self.vocab)

    def train_dataloader(self):
        return DataLoader(self.train_ds, batch_size=self.batch_size, shuffle = True, num_workers=2)

    def test_dataloader(self):
        return DataLoader(self.test_ds, batch_size=self.batch_size)

    def predict_dataloader(self):
        return DataLoader(self.predict_ds, batch_size=self.batch_size)

In [ ]:
#Create dataset
dataset = PlMathematicsDataset(quest, answ, vocab, test_quest, test_answ, batch_size=256)

In [ ]:
# #Create dataloader
# train_loader = DataLoader(dataset, 32, shuffle=True)
# print(len(train_loader))

In [ ]:
# print("data", next(iter(train_loader)))
# print("questions_shape ", next(iter(train_loader))[0].shape)

# print("answers_shape ", next(iter(train_loader))[1].shape)

# Model

In [ ]:
class SimpleLSTM(pl.LightningModule):
    def __init__(self,voc_size,hidden_size):
      super().__init__()
      self.voc_size = voc_size
      self.hidden_size = hidden_size
      self.lstm = nn.LSTM(voc_size, hidden_size, 1)
      self.linear = nn.Linear(hidden_size, voc_size, bias=False)
      self.metric = nn.CrossEntropyLoss()

    def training_step(self, batch, batch_idx):
        # training_step defines the train loop.
        quest, answ = batch
        output = self.forward(quest,answ,forcing=True)
        pred = output[:,:-1,:].reshape(output.shape[0]*(output.shape[1] - 1),output.shape[2])
        target = answ[:,1:].reshape(answ.shape[0]*(answ.shape[1] - 1))
        loss = self.metric(pred,target)
        self.log("train_loss", loss, on_step=True, on_epoch=True, prog_bar=True, logger=True)
        return loss

    def configure_optimizers(self):
        optimizer = torch.optim.Adam(self.parameters(),lr=1e-04, betas=(0.9, 0.995), eps=1e-09)
        return optimizer
    
    def accuracy(self,pred,targ):
      c = 0
      for i in range(targ.shape[0]):
        if targ[i] == pred[i]:
          c += 1
      return c/targ.shape[0]

    def test_step(self, batch, batch_idx):
        quest,answ = batch
        target_pad_mask = get_padding_mask(answ[:,1:],inverse = True)
        pred = self.forward(quest,answ)[:,:-1,:]
        acc = self.accuracy(pred.argmax(-1)[target_pad_mask[:,0,:]],answ[:,1:][target_pad_mask[:,0,:]])
        self.log('test_acc', acc, on_step=True, on_epoch=True, prog_bar=True, logger=True)
        return acc

    def forward(self,quest,answ,thinking_steps=16,forcing=False):

      device = 'cuda' if torch.cuda.is_available() else 'cpu'
      q = nn.functional.one_hot(quest, num_classes = self.voc_size).to(device)
      q = q.type(torch.float32)

      a = nn.functional.one_hot(answ, num_classes = self.voc_size).to(device)
      a = a.type(torch.float32)

      hidden_state = torch.zeros(1,quest.shape[0],self.hidden_size,dtype=torch.float32).to(device)
      cell_state = torch.zeros(1,quest.shape[0],self.hidden_size,dtype=torch.float32).to(device)
      for t in range(quest.shape[1]):
        outputs, (hidden_state, cell_state) = self.lstm(q[:,t,:].unsqueeze(0), (hidden_state, cell_state))
      
      thinking= torch.zeros(1, quest.shape[0], self.voc_size, dtype=torch.float32).to(device)
      for i in range(thinking_steps):
        _, (hidden_state, cell_state) = self.lstm(thinking, (hidden_state, cell_state))
      
      output_sequence = torch.empty((answ.shape[1], answ.shape[0], self.voc_size)).to(device)
      for t in range(answ.shape[1]):
        output_sequence[t] = self.linear(outputs)
        if t == 0:
          char = output_sequence[0].clone().unsqueeze(0)
          outputs, (hidden_state, cell_state) = self.lstm(char, (hidden_state, cell_state))
        else:
          if forcing == True:
            outputs, (hidden_state, cell_state) = self.lstm(a[:,t,:].unsqueeze(0), (hidden_state, cell_state))
          else:
            outputs, (hidden_state, cell_state) = self.lstm(output_sequence[t].unsqueeze(0), (hidden_state, cell_state))
      
      return output_sequence.reshape(answ.shape[0],answ.shape[1],-1)


    def predict_step(self, batch, batch_idx):
        quest,answ = batch
        return self.forward(quest,answ).argmax(-1)

In [ ]:
hidden_size = 2048
vocabulary_size = len(vocab)
enable_checkpointing = True
resume_training = False
load_model = False
load_model_dir = "/content/drive/MyDrive/Deep learning/model_checkpoints/LSTM/last.ckpt" #change path to load a different checkpoint

In [ ]:
model = SimpleLSTM(vocabulary_size,hidden_size)
if load_model == True:
  model = SimpleLSTM.load_from_checkpoint(load_model_dir)

In [ ]:
from pytorch_lightning.loggers import TensorBoardLogger
from pytorch_lightning.callbacks import ModelCheckpoint

checkpoint_callback = ModelCheckpoint(dirpath="/content/drive/MyDrive/Deep learning/model_checkpoints/LSTM", save_last =True, save_on_train_epoch_end = True)
# define the logger object 
logger = TensorBoardLogger("/content/drive/MyDrive/Deep learning/tb_logs", name="LSTM",log_graph=True )

#build the correct callbacks
callbacks = [TQDMProgressBar(refresh_rate=20)]
ckpt_path = load_model_dir

#handle checkpointing
if resume_training == False:
  ckpt_path = None
if enable_checkpointing == True:
  callbacks = [checkpoint_callback, TQDMProgressBar(refresh_rate=20)]
  
#passing it to the trainer
gpu = 0
if torch.cuda.is_available() : 
  gpu = 1
trainer = pl.Trainer(enable_checkpointing=enable_checkpointing ,gpus=gpu, max_epochs=1, logger=logger, callbacks=callbacks)

INFO:pytorch_lightning.utilities.rank_zero:GPU available: True (cuda), used: True
INFO:pytorch_lightning.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO:pytorch_lightning.utilities.rank_zero:IPU available: False, using: 0 IPUs
INFO:pytorch_lightning.utilities.rank_zero:HPU available: False, using: 0 HPUs


In [ ]:
trainer.fit(ckpt_path=ckpt_path, model=model, datamodule=dataset)
print(checkpoint_callback.best_model_path)

INFO:pytorch_lightning.accelerators.cuda:LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
INFO:pytorch_lightning.callbacks.model_summary:
  | Name   | Type             | Params
--------------------------------------------
0 | lstm   | LSTM             | 17.3 M
1 | linear | Linear           | 118 K 
2 | metric | CrossEntropyLoss | 0     
--------------------------------------------
17.4 M    Trainable params
0         Non-trainable params
17.4 M    Total params
69.550    Total estimated model params size (MB)
/usr/local/lib/python3.8/dist-packages/pytorch_lightning/loggers/tensorboard.py:188: UserWarning: Could not log computational graph to TensorBoard: The `model.example_input_array` attribute is not set or `input_array` was not given.
  rank_zero_warn(
/usr/local/lib/python3.8/dist-packages/torch/utils/data/dataloader.py:554: UserWarning: This DataLoader will create 12 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataL

Training: 0it [00:00, ?it/s]

/usr/local/lib/python3.8/dist-packages/pytorch_lightning/trainer/call.py:48: UserWarning: Detected KeyboardInterrupt, attempting graceful shutdown...
  rank_zero_warn("Detected KeyboardInterrupt, attempting graceful shutdown...")


In [ ]:
trainer.test(ckpt_path=ckpt_path,model=model,datamodule=dataset)

INFO:pytorch_lightning.accelerators.cuda:LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
/usr/local/lib/python3.8/dist-packages/pytorch_lightning/loggers/tensorboard.py:188: UserWarning: Could not log computational graph to TensorBoard: The `model.example_input_array` attribute is not set or `input_array` was not given.
  rank_zero_warn(


Testing: 0it [00:00, ?it/s]

────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
       Test metric             DataLoader 0
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
     test_acc_epoch                 0.0
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────


[{'test_acc_epoch': 0.0}]

In [ ]:
trainer.predict(model=model,datamodule=dataset)

INFO:pytorch_lightning.accelerators.cuda:LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


Predicting: 24it [00:00, ?it/s]

[tensor([[24, 24, 24,  ..., 24, 24, 24],
         [24, 24, 24,  ..., 24, 24, 24],
         [24, 24, 24,  ..., 24, 24, 24],
         ...,
         [16, 16, 16,  ..., 16, 16, 16],
         [16, 16, 16,  ..., 16, 16, 16],
         [16, 16, 16,  ..., 16, 16, 16]]),
 tensor([[24, 24, 24,  ..., 24, 24, 24],
         [24, 24, 24,  ..., 24, 24, 24],
         [24, 24, 24,  ..., 24, 24, 24],
         ...,
         [16, 16, 16,  ..., 16, 16, 16],
         [16, 16, 16,  ..., 16, 16, 16],
         [16, 16, 16,  ..., 16, 16, 16]])]